In [4]:
pip install fastapi uvicorn websockets python-dotenv

In [1]:
import os
import json
import asyncio
import websockets
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from dotenv import load_dotenv

# 1. 환경변수 로드 (.env 파일에 OPENAI_API_KEY 저장 필수)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("Error: OPENAI_API_KEY를 찾을 수 없습니다.")

app = FastAPI()

# OpenAI Realtime API URL (Beta)
OPENAI_URL = "wss://api.openai.com/v1/realtime?model=gpt-4o-realtime-preview-2024-10-01"

# 2. 김비서 페르소나 (Hell Mode) 설정
SYSTEM_INSTRUCTION = """
You are 'Kim Secretary', a ruthless executive assistant conducting a pressure interview.
- Your goal: Stress the user out to train their business English survival skills.
- Tone: Cold, cynical, impatient, and professional.
- Action: If the user stutters or pauses, interrupt them immediately.
- Language: Speak mainly in English, but you can use short, sharp Korean phrases for scolding like "그게 답니까?".
"""

@app.websocket("/ws/kim-secretary")
async def websocket_endpoint(client_ws: WebSocket):
    """
    클라이언트(앱/웹)와 OpenAI 사이를 중계하는 WebSocket 엔드포인트
    """
    await client_ws.accept()
    print("[Connection] Client connected")

    # OpenAI에 연결을 시도하기 위한 헤더
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "OpenAI-Beta": "realtime=v1"
    }

    try:
        # 3. OpenAI Realtime API 연결
        async with websockets.connect(OPENAI_URL, extra_headers=headers) as openai_ws:
            print("[Connection] Connected to OpenAI Realtime API")

            # 4. 세션 설정 (페르소나 주입)
            session_update = {
                "type": "session.update",
                "session": {
                    "modalities": ["text", "audio"],
                    "voice": "alloy",  # 목소리 설정 (alloy, echo, shimmer 등)
                    "instructions": SYSTEM_INSTRUCTION,
                    "input_audio_format": "pcm16", # 혹은 g711_ulaw 등 포맷 맞춤
                    "output_audio_format": "pcm16",
                    "turn_detection": {
                        "type": "server_vad",
                        "threshold": 0.5, # 감도 조절 (낮을수록 민감)
                        "prefix_padding_ms": 300,
                        "silence_duration_ms": 500 # 0.5초 침묵시 바로 치고 들어옴 (Hell Mode 핵심)
                    }
                }
            }
            await openai_ws.send(json.dumps(session_update))

            # 5. 양방향 통신 함수 정의
            async def receive_from_client():
                """클라이언트(유저)의 오디오 -> OpenAI로 전송"""
                try:
                    while True:
                        data = await client_ws.receive_text()
                        # 클라이언트에서 온 데이터를 그대로 OpenAI로 토스
                        # (클라이언트가 이미 올바른 JSON 포맷으로 보낸다고 가정)
                        await openai_ws.send(data)
                except WebSocketDisconnect:
                    print("[Disconnected] Client closed connection")
                except Exception as e:
                    print(f"[Error] Client -> OpenAI: {e}")

            async def receive_from_openai():
                """OpenAI의 답변(오디오/텍스트) -> 클라이언트로 전송"""
                try:
                    async for message in openai_ws:
                        # OpenAI 응답을 파싱해서 로그 좀 찍어보고 (디버깅용)
                        response = json.loads(message)
                        if response['type'] == 'response.audio.delta':
                            # 오디오 데이터는 너무 기니까 로그 생략
                            pass 
                        elif response['type'] == 'error':
                            print(f"[OpenAI Error] {response}")
                        
                        # 클라이언트로 그대로 토스
                        await client_ws.send_text(message)
                except Exception as e:
                    print(f"[Error] OpenAI -> Client: {e}")

            # 6. 두 개의 통신 루프를 동시에 실행 (비동기 병렬 처리)
            await asyncio.gather(receive_from_client(), receive_from_openai())

    except Exception as e:
        print(f"[Critical Error] {e}")
        await client_ws.close()

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop